In [1]:
import pandas as pd
import numpy as np

In [51]:
pmt_data = pd.read_csv("Claims Payment Data July 5.csv")
pmt_data = pmt_data[pmt_data["company"] != "the General"]
pmt_data = pmt_data.drop_duplicates()
pmt_data

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_20968\1956155234.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pmt_data = pd.read_csv("Claims Payment Data July 5.csv")


,adjuster_group,adjuster_user_id,check_delivery_method,check_delivery_type,check_id,check_number,check_status,claim_number,company,draft_amount,...,issuedByDept,issuedByDiv,issuedMonth,issuer_user_id,pay_to,payment_method,policy_state,reason_for_void,void_date,Record Count
0,Wilber,001wpu,Central Print,NaN,19348234,1151261056,Cleared,01-002-039717,Amfam,150.28,...,Subrogation Support Unit,Clms Shared Services Div,5,LROSS1,"KEIS George, LLP",Electronic funds transfer,IL,NaN,NaN,1
1,Wilber,001wpu,Central Print,NaN,19584882,5664066,Cleared,01-003-734195,Amfam,5752.73,...,NaN,NaN,6,GCLAPSP,Terry Burgos,Check,AZ,NaN,NaN,1
2,Wilber,001wpu,Central Print,First Class,19431726,5577733,Cleared,01-001-985518,Amfam,500.00,...,Subrogation Support Unit,Clms Shared Services Div,5,TVOS,Laura Garrison,Check,MO,NaN,NaN,1
3,Wilber,001wpu,Central Print,NaN,19620632,1151275842,Cleared,01-001-640219,Amfam,136.92,...,Subrogation Support Unit,Clms Shared Services Div,6,LROSS1,KEIS GEORGE LLP,Electronic funds transfer,IN,NaN,NaN,1
4,Wilber,001wpu,Central Print,First Class,19205037,5472769,Cleared,01-003-593730,Amfam,500.00,...,Subrogation Support Unit,Clms Shared Services Div,5,AMACE,Erick Vizguerra Hernandez And Guadalupe Tapia,Check,WA,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543681,APD Operations 1,LMCNAL,NaN,NaN,19454946,T0000010393,Cleared,01-004-616115,Amfam,180.00,...,Auto APD Srvcs Rental Unit 1,Clms Auto Adjusting Div,6,LROUGH,American Family Insurance,Manual check,KS,NaN,NaN,1
543682,APD Operations 1,LMCNAL,NaN,NaN,19143454,T0000010298,Cleared,01-004-616115,Amfam,390.00,...,Auto APD Srvcs Rental Unit 1,Clms Auto Adjusting Div,5,LROUGH,American Family Insurance,Manual check,KS,NaN,NaN,1
543683,AMFAM View Group,SBRIDG,Central Print,NaN,19211614,1121195807,Cleared,01-004-326771,Amfam,418.12,...,Auto APD Field Tm 2 Unit 6,Clms Auto Adjusting Div,5,AHLINS,CARR AUTO BODY,Electronic funds transfer,OR,NaN,NaN,1
543684,AMFAM View Group,SBRIDG,Central Print,NaN,19206573,2000162457,Cleared,01-004-326771,Amfam,463.63,...,Auto PIP West Team 6,Clms Auto Adjusting Div,5,NBOUSK,BEESON CHIROPRACTIC CENTER,Electronic funds transfer,OR,NaN,NaN,1


In [45]:
def month_counts(months):
    values = []
    for month in months:
        month_data = pmt_data[pmt_data["issuedMonth"] == month][pmt_data["check_status"] != ("Pending void")][pmt_data["check_status"] != "Transferred"]
        overnight_count = len(month_data[month_data["check_delivery_type"] == "Overnight"])
        voided_count = len(month_data[month_data["check_status"] == "Voided"]) 
        
        eft_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"])
        eft_check_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] == "Check"])
        debit_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] == "Debit Card"])
        direct_deposit_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] == "Direct Deposit"])
        virtual_card_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] == "Virtual Card"])
        venmo_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] == "Venmo"])
        paypal_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] == "PayPal"])
        eft_null = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] != "PayPal"][month_data["ipMethod"] != "Venmo"][month_data["ipMethod"] != "Direct Deposit"][month_data["ipMethod"] != "Virtual Card"][month_data["ipMethod"] != "Debit Card"][month_data["ipMethod"] != "Check"]["ipMethod"])
        sum_count = eft_check_count + direct_deposit_count + debit_count + virtual_card_count + venmo_count + paypal_count + eft_null
        
        stopped_count = len(month_data[month_data["check_status"] == "Stopped"])
        
        bool_series = pd.notnull(month_data["check_delivery_method"])
        paper_data = month_data[bool_series]
        paper_data = paper_data[paper_data["check_delivery_type"] != "Overnight"][paper_data["payment_method"] == "Check"][paper_data["ipMethod"] != ("PayPal" or "Venmo" or "Direct Deposit" or "Virtual Card")]
        paper_count = len(paper_data)
        
        cost = "$" + str((overnight_count * 56) + (voided_count * 15) + (paper_count) + (eft_check_count * 1.23) + (debit_count  * 1.1) + (direct_deposit_count * 1.1) + (virtual_card_count * 0) + (venmo_count * 1 ) + (paypal_count * 1) + (eft_null * 0)) 
        values.append([month, paper_count, overnight_count, voided_count, stopped_count, eft_check_count, debit_count, direct_deposit_count, virtual_card_count, venmo_count, paypal_count, eft_null, eft_count, cost])
    df =  pd.DataFrame(values, columns=["Month", "Paper Check Count ($1)",  "Overnight Check Count ($56)", "Voided Check Count ()", "Stopped Check Count", "EFT Check Count", "EFT Debit Card Count","EFT Direct Deposit Count", "EFT Virtual Card Count", "EFT Venmo Count", "EFT PayPal Count", "EFT NULL count", "Total EFT Payment Count", "Total Cost"]).set_index("Month")
    return df

In [53]:
results = month_counts([5,6])

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_20968\4000581195.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  month_data = pmt_data[pmt_data["issuedMonth"] == month][pmt_data["check_status"] != ("Pending void")][pmt_data["check_status"] != "Transferred"]
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_20968\4000581195.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  eft_check_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] == "Check"])
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_20968\4000581195.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  debit_count = len(month_data[month_data["payment_method"] == "Electronic funds transfer"][month_data["ipMethod"] == "Debit Card"])
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_20968\4000581195.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  

In [57]:
results.to_csv("May-June Payments Delivery Costs.csv", index = False )